# Entity Resolution Application using PySpark

Entity resolution, also known as record linkage or deduplication, is a data integration process that identifies
and links records that refer to the same real-world entity across diverse data sources. The goal is to
reconcile and merge information about entities, such as individuals or businesses, even when they are
represented inconsistently or incompletely in different datasets. Entity resolution involves comparing and
analyzing attributes like names, addresses, and other identifying information to determine the likelihood
of a match. This process is crucial in various domains, including customer relationship management,
healthcare, finance, and law enforcement, where accurate and consolidated data is essential. Advanced
techniques, such as probabilistic matching and machine learning algorithms, are often employed to
enhance the accuracy and efficiency of entity resolution in handling large and complex datasets.

Lab Exercises:

1) Develop a PySpark script to clean and preprocess data before performing entity resolution.Include steps like tokenization and normalization.

2) Implement a PySpark program that computes similarity scores between records using a chosen similarity metric.

3)Implement a PySpark program to evaluate the precision, recall, and F1-score of an entity resolution model.

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions

In [36]:
# spark = SparkSession.builder.appName('Lab 3').getOrCreate()
spark = SparkSession.builder.getOrCreate()

## Setting up the data and analyzing it¶

In [42]:
df = spark.read.option("recursiveFileLookup","true").csv("donation/block_1.csv")
# df.show(3)

df = spark.read.option("header","true").option("nullValue","?").option("inferSchema","true").csv("donation//block_1.csv")
df.printSchema()
df.show(5)

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        null|         1.0|        null|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|       

In [18]:
from pyspark.sql.functions import col
df.groupBy("is_match").count().orderBy(col("count").desc()).show()



df.createOrReplaceTempView("linkage")

spark.sql("""
SELECT is_match, COUNT(*) AS cnt
FROM linkage
GROUP BY linkage.is_match 
ORDER BY cnt DESC"""
).show()

+--------+------+
|is_match| count|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+

+--------+------+
|is_match|   cnt|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+



In [5]:
df.count()

574913

In [6]:
df.cache()

DataFrame[id_1: int, id_2: int, cmp_fname_c1: double, cmp_fname_c2: double, cmp_lname_c1: double, cmp_lname_c2: double, cmp_sex: int, cmp_bd: int, cmp_bm: int, cmp_by: int, cmp_plz: int, is_match: boolean]

## Fast Summary Statistics, Plotting and Reshaping DataFrames

In [44]:
summary = df.describe()
summary.select("summary","cmp_fname_c1","cmp_fname_c2").show()

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|            574811|             10325|
|   mean|0.7127592938253411|0.8977586763518969|
| stddev|0.3889286452463531|0.2742577520430532|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [45]:
matches = df.where("is_match = true")
match_summary = matches.describe()
misses = df.filter(col('is_match') == False)
miss_summary = misses.describe()

In [29]:
summary_p = summary.toPandas()
summary_p.head()

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,574913,574913,574811,10325,574913,239,574913,574851,574851,574851,573618
1,mean,33271.962171667714,66564.6636865056,0.7127592938253411,0.8977586763518969,0.3155724578100624,0.3269155414552904,0.9550923357099248,0.22475563232907309,0.4886361857246487,0.22266639529199742,0.0054949461139643455
2,stddev,23622.669425933756,23642.00230967228,0.3889286452463531,0.2742577520430532,0.3342494687554245,0.3783092020540671,0.20710152240504442,0.4174216587235557,0.4998712818281637,0.4160365041645591,0.07392402321301972
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99894,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [30]:
summary_p.shape

(5, 12)

In [31]:
summary_p = summary_p.set_index('summary').transpose().reset_index()

summary_p.head()

summary,index,count,mean,stddev,min,max
0,id_1,574913,33271.962171667714,23622.669425933756,1,99894
1,id_2,574913,66564.6636865056,23642.00230967228,6,100000
2,cmp_fname_c1,574811,0.7127592938253411,0.3889286452463531,0.0,1.0
3,cmp_fname_c2,10325,0.8977586763518969,0.2742577520430532,0.0,1.0
4,cmp_lname_c1,574913,0.3155724578100624,0.3342494687554245,0.0,1.0


In [47]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
# summary_p = summary_p.rename(columns={'index':'field'})
# summary_p = summary_p.rename_axis(None,axis=1)
# summary_p.shape

KeyError: "None of ['summary'] are in the columns"